In [15]:
import json
import numpy as np
import pygad

def idm_model(params, initial_conditions, time_steps, lead_vehicle_speeds, lead_vehicle_positions, vehicle_length):
    a_max, s_0, T, b = params
    v0 = 33.3  # m/s
    delta = 4

    velocities = [initial_conditions['v']]
    gaps = []
    accelerations = []

    if not lead_vehicle_positions or not lead_vehicle_speeds:  # 如果前车位置或速度列表为空
        for t in range(1, len(time_steps)):
            v = velocities[-1]
            a = a_max * (1 - (v / v0) ** delta)  # 自由流加速度
            v_next = v + a * (time_steps[t] - time_steps[t - 1])
            velocities.append(v_next)
            gaps.append(0)  # 用0填充车距
            accelerations.append(a)
        
        # 直接补齐加速度和速度列表的最后一个元素
        velocities.extend([velocities[-1]] * (len(time_steps) - len(velocities)))
        accelerations.extend([accelerations[-1]] * (len(time_steps) - 1 - len(accelerations)))
        
        return velocities, gaps, accelerations

    for t in range(1, len(time_steps)):
        v = velocities[-1]
        s = initial_conditions['s'] + sum(velocities[:-1]) * (time_steps[t] - time_steps[t - 1])

        s_lead = lead_vehicle_positions[t - 1] if t - 1 < len(lead_vehicle_positions) else lead_vehicle_positions[-1]
        gap = s_lead - s - vehicle_length
        delta_v = v - lead_vehicle_speeds[t - 1]
        s_star = s_0 + v * T + (v * delta_v) / (2 * np.sqrt(a_max * b))
        
        if lead_vehicle_speeds[t - 1] == float('inf'):  # 自由驾驶状态
            a = a_max * (1 - (v / v0) ** delta)
        else:
            a = a_max * (1 - (v / v0) ** delta - (s_star / gap) ** 2)

        v_next = v + a * (time_steps[t] - time_steps[t - 1])

        velocities.append(v_next)
        gaps.append(gap)
        accelerations.append(a)

    # 直接补齐加速度和速度列表的最后一个元素
    velocities.extend([velocities[-1]] * (len(time_steps) - len(velocities)))
    accelerations.extend([accelerations[-1]] * (len(time_steps) - 1 - len(accelerations)))

    return velocities, gaps, accelerations

def calculate_mse(real, predicted):
    min_length = min(len(real), len(predicted))
    real = real[:min_length]
    predicted = predicted[:min_length]
    return np.mean((np.array(real) - np.array(predicted)) ** 2)

def fitness_function(ga_instance, solution, solution_idx):
    user_data = ga_instance.user_data
    initial_conditions_before, initial_conditions_after, time_steps_before, time_steps_after, lead_vehicle_speeds_before, lead_vehicle_speeds_after, lead_vehicle_positions_before, lead_vehicle_positions_after, vehicle_length_before, vehicle_length_after, real_values_before, real_values_after = user_data

    velocities_before, gaps_before, accelerations_before = idm_model(solution, initial_conditions_before, time_steps_before, lead_vehicle_speeds_before, lead_vehicle_positions_before, vehicle_length_before)
    velocities_after, gaps_after, accelerations_after = idm_model(solution, initial_conditions_after, time_steps_after, lead_vehicle_speeds_after, lead_vehicle_positions_after, vehicle_length_after)

    mse_gap_before = calculate_mse(real_values_before['gaps'], gaps_before)
    mse_vel_before = calculate_mse(real_values_before['velocities'], velocities_before)
    mse_acc_before = calculate_mse(real_values_before['accelerations'], accelerations_before)

    mse_gap_after = calculate_mse(real_values_after['gaps'], gaps_after)
    mse_vel_after = calculate_mse(real_values_after['velocities'], velocities_after)
    mse_acc_after = calculate_mse(real_values_after['accelerations'], accelerations_after)

    mse_total = mse_gap_before + mse_vel_before + mse_acc_before + mse_gap_after + mse_vel_after + mse_acc_after

    return -mse_total  # pyGAD maximizes fitness, so return the negative MSE

def optimize_all_entries(idm_file_path, real_file_path, output_file):
    with open(idm_file_path, 'r') as idm_file, open(real_file_path, 'r') as real_file:
        idm_lines = idm_file.readlines()
        real_lines = real_file.readlines()
        
        if len(idm_lines) != len(real_lines):
            raise ValueError("IDM文件和真实值文件中的条目数不一致")
        
        idm_data = [json.loads(line.strip()) for line in idm_lines]
        real_data = [json.loads(line.strip()) for line in real_lines]

    optimized_results = []

    for idm_entry, real_entry in zip(idm_data, real_data):
        initial_conditions_before = {
            'v': idm_entry['vehicle_initial_velocity'],
            's': idm_entry['vehicle_initial_position']
        }
        initial_conditions_after = {
            'v': idm_entry['vehicle_post_change_velocity'],
            's': idm_entry['vehicle_post_change_position']
        }

        time_steps_before = [t for t in idm_entry['timestamps'] if t <= idm_entry['after_change_timestamp']]
        time_steps_after = [t for t in idm_entry['timestamps'] if t > idm_entry['after_change_timestamp']]

        lead_vehicle_speeds_before = [speed * 0.44704 for speed in idm_entry['before_preceding_velocities']]  # mph to m/s
        lead_vehicle_speeds_after = [speed * 0.44704 for speed in idm_entry['after_preceding_velocities']]  # mph to m/s

        lead_vehicle_positions_before = idm_entry.get('before_preceding_positions', [])
        lead_vehicle_positions_after = idm_entry.get('after_preceding_positions', [])

        vehicle_length_before = idm_entry['v_length']
        vehicle_length_after = idm_entry['v_length']

        real_values_before = {
            'gaps': real_entry['gap_before_change'],
            'velocities': real_entry['vehicle_speeds'][:len(time_steps_before)],
            'accelerations': real_entry['vehicle_accelerations'][:len(time_steps_before)]
        }
        real_values_after = {
            'gaps': real_entry['gap_after_change'],
            'velocities': real_entry['vehicle_speeds'][len(time_steps_before):],
            'accelerations': real_entry['vehicle_accelerations'][len(time_steps_before):]
        }

        user_data = (initial_conditions_before, initial_conditions_after, time_steps_before, time_steps_after, lead_vehicle_speeds_before, lead_vehicle_speeds_after, lead_vehicle_positions_before, lead_vehicle_positions_after, vehicle_length_before, vehicle_length_after, real_values_before, real_values_after)

        gene_space = [
            {'low': 0.1, 'high': 5.0},  # a_max
            {'low': 0.1, 'high': 5.0},  # s_0
            {'low': 0.1, 'high': 5.0},  # T
            {'low': 0.1, 'high': 5.0}   # b
        ]

        ga_instance = pygad.GA(
            num_generations=50,
            num_parents_mating=5,
            fitness_func=fitness_function,
            sol_per_pop=10,
            num_genes=4,
            gene_space=gene_space,
            gene_type=float,
            parent_selection_type="sss",
            keep_parents=2,
            crossover_type="single_point",
            mutation_type="random",
            mutation_percent_genes=10
        )

        # 设置用户数据
        ga_instance.user_data = user_data

        ga_instance.run()

        solution, solution_fitness, solution_idx = ga_instance.best_solution()
        a_max, s_0, T, b = solution

        optimized_result = {
            'vehicle_id': idm_entry['vehicle_id'],
            'before_lane_id': idm_entry['before_lane_id'],
            'after_lane_id': idm_entry['after_lane_id'],
            'before_preceding_id': idm_entry['before_preceding_id'],
            'after_preceding_id': idm_entry['after_preceding_id'],
            'a_max': a_max,
            's_0': s_0,
            'T': T,
            'b': b,
            'mse': -solution_fitness
        }

        optimized_results.append(optimized_result)

        if len(optimized_results) % 10 == 0:
            print(f"Processed {len(optimized_results)} lane changes")

    with open(output_file, 'w') as f:
        for result in optimized_results:
            json.dump(result, f)
            f.write('\n')

    print(f"Total optimized entries: {len(optimized_results)}")

# 文件路径
idm_file_path = 'trajectories-0400-0415-idm_modeling_data.json'
real_file_path = 'trajectories-0400-0415-real_values_data.json'
output_file = 'trajectories-0400-0415-optimized_idm_parameters.json'

# 优化所有数据
optimize_all_entries(idm_file_path, real_file_path, output_file)

C:\Users\Summer\AppData\Local\Temp\ipykernel_19816\2576211543.py:36: RuntimeWarning: overflow encountered in scalar multiply
  s_star = s_0 + v * T + (v * delta_v) / (2 * np.sqrt(a_max * b))
C:\Users\Summer\AppData\Local\Temp\ipykernel_19816\2576211543.py:41: RuntimeWarning: overflow encountered in scalar power
  a = a_max * (1 - (v / v0) ** delta - (s_star / gap) ** 2)
C:\Users\Summer\AppData\Local\Temp\ipykernel_19816\2576211543.py:36: RuntimeWarning: invalid value encountered in scalar add
  s_star = s_0 + v * T + (v * delta_v) / (2 * np.sqrt(a_max * b))
C:\Users\Summer\AppData\Local\Temp\ipykernel_19816\2576211543.py:59: RuntimeWarning: overflow encountered in square
  return np.mean((np.array(real) - np.array(predicted)) ** 2)
index 0 is out of bounds for axis 0 with size 0
Traceback (most recent call last):
  File "C:\anaconda\Lib\site-packages\pygad\pygad.py", line 2361, in best_solution
    best_match_idx = numpy.where(
                     ^^^^^^^^^^^^
IndexError: index 0 is o

IndexError: index 0 is out of bounds for axis 0 with size 0